In [1]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
fashion = pd.read_csv("Amazon Fashion.csv")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
fashion.head(1)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,"Aqualogica Glow+ Dewy Sunscreen SPF 50 PA+++ For UVA/B & Blue Light Protection, for Glowing & Well Protected Skin, Cream, 50G",stores,Amazon Fashion,https://m.media-amazon.com/images/I/51TSC6UogxL._AC_UL320_.jpg,https://www.amazon.in/Aqualogica-Sunscreen-Protection-Glowing-Protected/dp/B09TPFTJNN/ref=sr_1_49?qid=1679212359&s=apparel&sr=1-49,4.2,"3,628",₹351,₹399


In [5]:
fashion.isna().sum()

,0
name,0
main_category,0
sub_category,0
image,0
link,0
ratings,206
no_of_ratings,206
discount_price,166
actual_price,11


In [7]:
fashion.duplicated().sum()

0

In [8]:
fashion['#keywords'] = fashion['name'].str.lower()

In [9]:
fashion.head(2)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Aqualogica Glow+ Dewy Sunscreen SPF 50 PA+++ For UVA/B & Blue Light Protection, for Glowing & Well Protected Skin, Cream, 50G",stores,Amazon Fashion,https://m.media-amazon.com/images/I/51TSC6UogxL._AC_UL320_.jpg,https://www.amazon.in/Aqualogica-Sunscreen-Protection-Glowing-Protected/dp/B09TPFTJNN/ref=sr_1_49?qid=1679212359&s=apparel&sr=1-49,4.2,"3,628",₹351,₹399,"aqualogica glow+ dewy sunscreen spf 50 pa+++ for uva/b & blue light protection, for glowing & well protected skin, cream, 50g"
1,MARVIK Soft Silicone Adjustable Band Strap Compatible for Oneplus Smart Band & Oppo Smart Band,stores,Amazon Fashion,https://m.media-amazon.com/images/I/51vYDop04SL._AC_UL320_.jpg,https://www.amazon.in/MARVIK-Silicone-Adjustable-Compatible-Oneplus/dp/B0B7J1QG1F/ref=sr_1_50?qid=1679212359&s=apparel&sr=1-50,3.9,291,₹249,₹999,marvik soft silicone adjustable band strap compatible for oneplus smart band & oppo smart band


In [10]:
fashion['#keywords']

,#keywords
0,"aqualogica glow+ dewy sunscreen spf 50 pa+++ for uva/b & blue light protection, for glowing & well protected skin, cream, 50g"
1,marvik soft silicone adjustable band strap compatible for oneplus smart band & oppo smart band
2,108 panchatantra stories for children (illustrated) - story book for kids - moral stories - bedtime stories - 3 years to 1...
3,black and white and blue: adult cinema from the victorian age to the vcr
4,safari 15 ltrs sea blue casual/school/college backpack (daypackneo15cbseb)
...,...
2347,"lakme lip love gelato lip balm - pink, bubblegum, 4.5 grams"
2348,sattva classy.elegant.stylish classic xxxl bean bag filled with beans - grey
2349,fastrack glitch analog rose gold dial women's watch-68026wm01
2350,nainvish women/girl's pure cotton paisley printed kaftan top with pant (sd228-229_kb)


In [11]:
fashion.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price', '#keywords'],
      dtype='object')

In [12]:
print(fashion['main_category'].value_counts())
print(fashion['sub_category'].value_counts())

main_category
stores    2352
Name: count, dtype: int64
sub_category
Amazon Fashion    2352
Name: count, dtype: int64


In [13]:
fashion.drop(['main_category', 'sub_category'], axis=1, inplace=True)

In [14]:
fashion

,name,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Aqualogica Glow+ Dewy Sunscreen SPF 50 PA+++ For UVA/B & Blue Light Protection, for Glowing & Well Protected Skin, Cream, 50G",https://m.media-amazon.com/images/I/51TSC6UogxL._AC_UL320_.jpg,https://www.amazon.in/Aqualogica-Sunscreen-Protection-Glowing-Protected/dp/B09TPFTJNN/ref=sr_1_49?qid=1679212359&s=apparel&sr=1-49,4.2,"3,628",₹351,₹399,"aqualogica glow+ dewy sunscreen spf 50 pa+++ for uva/b & blue light protection, for glowing & well protected skin, cream, 50g"
1,MARVIK Soft Silicone Adjustable Band Strap Compatible for Oneplus Smart Band & Oppo Smart Band,https://m.media-amazon.com/images/I/51vYDop04SL._AC_UL320_.jpg,https://www.amazon.in/MARVIK-Silicone-Adjustable-Compatible-Oneplus/dp/B0B7J1QG1F/ref=sr_1_50?qid=1679212359&s=apparel&sr=1-50,3.9,291,₹249,₹999,marvik soft silicone adjustable band strap compatible for oneplus smart band & oppo smart band
2,108 Panchatantra Stories for Children (Illustrated) - Story Book for Kids - Moral Stories - Bedtime Stories - 3 Years to 1...,https://m.media-amazon.com/images/I/81VJ+MAc7YL._AC_UL320_.jpg,https://www.amazon.in/108-Panchatantra-Stories-Illustrated-children/dp/9390292921/ref=sr_1_51?qid=1679212359&s=apparel&sr=1-51,4.4,"2,950",₹125,₹160,108 panchatantra stories for children (illustrated) - story book for kids - moral stories - bedtime stories - 3 years to 1...
3,Black and White and Blue: Adult Cinema From the Victorian Age to the VCR,https://m.media-amazon.com/images/I/51RIhaW3t1L._AC_UL320_.jpg,https://www.amazon.in/Black-White-Blue-Cinema-Victorian-ebook/dp/B001TV00YK/ref=sr_1_52?qid=1679212359&s=apparel&sr=1-52,4.2,8,NaN,₹584.16,black and white and blue: adult cinema from the victorian age to the vcr
4,SAFARI 15 Ltrs Sea Blue Casual/School/College Backpack (DAYPACKNEO15CBSEB),https://m.media-amazon.com/images/I/61kmCas5OCL._AC_UL320_.jpg,https://www.amazon.in/SAFARI-Ltrs-Casual-Backpack-DAYPACKNEO15CBSEB/dp/B07Q7CNPMV/ref=sr_1_53?qid=1679212359&s=apparel&sr=1-53,4.0,"17,985",₹299,₹799,safari 15 ltrs sea blue casual/school/college backpack (daypackneo15cbseb)
...,...,...,...,...,...,...,...,...
2347,"Lakme Lip Love Gelato Lip Balm - Pink, Bubblegum, 4.5 grams",https://m.media-amazon.com/images/I/61lc+uAfF0L._AC_UL320_.jpg,https://www.amazon.in/LAKM%C3%89-Love-Gelato-Chapstick-Bubblegum/dp/B08HSST61W/ref=sr_1_18860?qid=1679212566&s=apparel&sr=1-18860,4.1,515,₹148,₹199,"lakme lip love gelato lip balm - pink, bubblegum, 4.5 grams"
2348,SATTVA Classy.Elegant.Stylish Classic XXXL Bean Bag Filled with Beans - Grey,https://m.media-amazon.com/images/I/71zaq78pO8L._AC_UL320_.jpg,https://www.amazon.in/Sattva-Classic-Filled-Beans-Colour/dp/B07T7NTPHB/ref=sr_1_18861?qid=1679212566&s=apparel&sr=1-18861,4.2,10,"₹2,170","₹4,299",sattva classy.elegant.stylish classic xxxl bean bag filled with beans - grey
2349,Fastrack Glitch Analog Rose Gold Dial Women's Watch-68026WM01,https://m.media-amazon.com/images/I/61B+fqeyzjL._AC_UL320_.jpg,https://www.amazon.in/Fastrack-Glitch-Analog-Womens-Watch-68026WM01/dp/B09J12746Z/ref=sr_1_18862?qid=1679212566&s=apparel&sr=1-18862,3.5,16,NaN,"₹5,595",fastrack glitch analog rose gold dial women's watch-68026wm01
2350,NAINVISH Women/Girl's Pure Cotton Paisley Printed Kaftan Top with Pant (SD228-229_KB),https://m.media-amazon.com/images/I/81qATlsYzkL._AC_UL320_.jpg,https://www.amazon.in/NAINVISH-Cotton-Paisley-Printed-SD228_KB-L/dp/B09PBXN59H/ref=sr_1_18863?qid=1679212566&s=apparel&sr=1-18863,3.4,318,₹499,"₹2,599",nainvish women/girl's pure cotton paisley printed kaftan top with pant (sd228-229_kb)


In [15]:
fashion['#keywords'] = fashion['#keywords'].str.replace('''[^\w\d\s]''','', regex = True)

In [16]:
fashion['#keywords']

,#keywords
0,aqualogica glow dewy sunscreen spf 50 pa for uvab blue light protection for glowing well protected skin cream 50g
1,marvik soft silicone adjustable band strap compatible for oneplus smart band oppo smart band
2,108 panchatantra stories for children illustrated story book for kids moral stories bedtime stories 3 years to 1
3,black and white and blue adult cinema from the victorian age to the vcr
4,safari 15 ltrs sea blue casualschoolcollege backpack daypackneo15cbseb
...,...
2347,lakme lip love gelato lip balm pink bubblegum 45 grams
2348,sattva classyelegantstylish classic xxxl bean bag filled with beans grey
2349,fastrack glitch analog rose gold dial womens watch68026wm01
2350,nainvish womengirls pure cotton paisley printed kaftan top with pant sd228229_kb


In [17]:
stemmer = PorterStemmer()

In [18]:
def stemming(text):
    words = []
    for word in text.split(' '):
        words.append(stemmer.stem(word))
    return ' '.join(words)

In [19]:
fashion['#keywords'] = fashion['#keywords'].apply(stemming)

In [20]:
fashion['#keywords']

,#keywords
0,aqualogica glow dewi sunscreen spf 50 pa for uvab blue light protect for glow well protect skin cream 50g
1,marvik soft silicon adjust band strap compat for oneplu smart band oppo smart band
2,108 panchatantra stori for children illustr stori book for kid moral stori bedtim stori 3 year to 1
3,black and white and blue adult cinema from the victorian age to the vcr
4,safari 15 ltr sea blue casualschoolcolleg backpack daypackneo15cbseb
...,...
2347,lakm lip love gelato lip balm pink bubblegum 45 gram
2348,sattva classyelegantstylish classic xxxl bean bag fill with bean grey
2349,fastrack glitch analog rose gold dial women watch68026wm01
2350,nainvish womengirl pure cotton paisley print kaftan top with pant sd228229_kb


In [21]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english', dtype = np.uint8)

In [22]:
cv.fit(fashion['#keywords'])

CountVectorizer(dtype=<class 'numpy.uint8'>, max_features=5000,
                stop_words='english')

In [23]:
vector = cv.transform(fashion['#keywords']).toarray()

In [24]:
vector.shape

(2352, 5000)

In [25]:
similarity = cosine_similarity(vector)

In [26]:
del(vector)
fashion.drop(['#keywords'], axis=1, inplace=True)

In [27]:
similarity.shape

(2352, 2352)

In [28]:
similarity[0]

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.07071068])

In [29]:
fashion['name'].sample(10, random_state=5)

,name
519,Fastrack Trendies Analog Pink Dial Women's Watch-NL68006PP04/NP68006PP04
1158,Fastrack Stunners 3.0 Analog Pink Dial Women's Watch-6267SM02
2204,Matrix Duet Signature Cut Glass Analog Wrist Watch (Pack of 2)
1920,Carrylux Quilted Design Sling Bags For Woman's
1680,FRIJVEN Solid 15 Cms Cosmetic Pouch (Standard_Cosmetic Bag_Navy Blue)
2235,"WOW Skin Science Retinol Face Cream - Oil Free, Quick Absorbing - For All Skin Types - No Parabens, Silicones, Color, Mine..."
1432,GaxQuly Travel Toiletry Waterproof Bag/Cosmatic Bag/Hanging Side Open Toiletry Bag/Extra Large Travel Makeup Organiser Bag...
557,Hemt Analog Black Dial Men's Watch-HM-GR092-BLK-CH
1709,Red Chief Leather with Lace Formal Shoes for Men RC3499
2152,"Namyaa Natural Lip Serum/Balm/Lightener/Moisturizer For Lip Lightening/Brightening/Toning/Moisturizing, 30 g"


In [31]:
product = "Fastrack Trendies Analog Pink Dial Women's Watch-NL68006PP04/NP68006PP04"
fashion[fashion['name'] == product]

,name,image,link,ratings,no_of_ratings,discount_price,actual_price
519,Fastrack Trendies Analog Pink Dial Women's Watch-NL68006PP04/NP68006PP04,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/719zgp3nwnL._AC_UL320_.jpg,https://www.amazon.in/Fastrack-Trendies-Analog-Womens-Watch-68006PP04/dp/B079VTRJJ4/ref=sr_1_2344?qid=1679212387&s=apparel&sr=1-2344,4.2,876,NaN,"₹2,195"


In [32]:

pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [33]:

pickle.dump(fashion, open('data.pkl', 'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')